https://www.kaggle.com/rtatman/get-body-text-of-e-mails/data

In [1]:
import os
import re
import csv
import operator
import collections
import numpy as np
from collections import Counter


In [2]:
# split word to ( , )

train_dir = 'fraud_data'
emails = [os.path.join(train_dir,f) for f in os.listdir(train_dir)]

all_words = []
for email in emails[:1000]:
    with open(email, encoding = 'ISO-8859-1') as m:
        a = []
        for i,line in enumerate(m):
            if i != 1:
                words_ = line
                pattern3 = re.compile("[^\w\d]+")
                words_ = pattern3.sub(' ',words_)
                words = words_.split()
                for j in range(len(words)):
                    a.append(words[j])
    all_words.append(a)
print ('size of letter :', len(all_words))

size of letter : 1000


In [3]:
import numpy as np

np.save('all_words.npy', all_words)

In [4]:
cut_programs = np.load('all_words.npy')

In [5]:
len(cut_programs)

1000

In [6]:
# generate dictionary

word_dict = dict()

In [7]:
def add_word_dict(w):
    if not w in word_dict:
        word_dict[w] = 1
    else:
        word_dict[w] += 1

In [8]:
for letter in cut_programs:
    for w in letter:
        add_word_dict(w)

In [9]:
import operator

word_dict = sorted(word_dict.items(), key=operator.itemgetter(1), reverse=True)
print ('size of word :', len(word_dict))

size of word : 23700


In [10]:
# choose word by frequence 10 < f

VOC_SIZE = 3000
VOC_START = 50

voc_dict = word_dict[VOC_START:VOC_START+VOC_SIZE]
print(voc_dict[:10])
np.save('voc_dict.npy', voc_dict)

[('our', 1184), ('fund', 1151), ('any', 1115), ('A', 1105), ('Mr', 1096), ('all', 1085), ('contact', 1051), ('business', 1038), ('an', 1024), ('transaction', 990)]


In [11]:
voc_dict = np.load('voc_dict.npy')

In [12]:
# word filter

example_doc = []

for letter in cut_programs:
    example_line = ''
    for w in letter:
        if w in voc_dict:
            example_line += w+' '
        
    example_doc.append(example_line)

print( example_doc[0] )

name Sani Abacha 2Cthe former first also wife former late head state Federal Republic Nigeria Gen Abacha whose sudden death came 8th June Since death 2CI been into present civilian administration been physical security country My son Mohammed under before federal high court Nigeria an he did commit he has been released since based certain he allowed move around any where has been own town only also been placed under government As widow so lost confidence anybody within country You must heard over reports internet various huge sums deposited husband different security firms abroad some companies give up their disclosed our lodged there many out right In fact total sum discovered Government so far tune 700 dollars And they make poor life 2EI got contacts through personal research 2Cand out decided reach through medium 2EI give more information regard soon reply repose great confidence hence due security placed day day affairs cannot visit embassy so why decided contact hope betray confid

### keywords ectraction

In [13]:
import math
import nltk
from textblob import TextBlob as tb

nltk.download('punkt')

def tf(word, blob):
    return blob.words.count(word) / len(blob.words)

def n_containing(word, bloblist):
    return sum(1 for blob in bloblist if word in blob)

def idf(word, bloblist):
    return math.log(len(bloblist) / (1 + n_containing(word, bloblist)))

def tfidf(word, blob, bloblist):
    return tf(word, blob) * idf(word, bloblist)

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\nems\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [14]:
bloblist = []
for i in range(len(example_doc)):
    blob = tb(example_doc[i])
    bloblist.append(blob)
print (len(bloblist))

1000


In [15]:
kw_n = 10

key_list = []
for i, blob in enumerate(bloblist):
    scores = {word: tfidf(word, blob, bloblist) for word in blob.words}
    sorted_words = sorted(scores.items(), key=lambda x: x[1], reverse=True)
    tmp = ""
    for word, score in sorted_words[ :kw_n]:
        tmp += word + ' '
    key_list.append(tmp)

In [16]:
key_list

['2EI Abacha security confidence federal whose placed abroad Federal yours ',
 'lord Lord 92t GOD Vault church MRS Any Reply USE ',
 'br 3D 92t GOD lord 3D0 Lord 10px Vault style ',
 'Plc Engr 5m Lagos avoid Get Nigeria aware Now development ',
 'multi MIME Content plain message printable quoted 8859 iso Encoding ',
 'Miss COAST Content IVORY real Disposition human wish activities sudden ',
 'br Miss COAST IVORY real Disposition human wish activities sudden ',
 'Netherlands camp united May 205 president refugee Republic sincerely President ',
 'lord Lord church Contact Blessed always Bible He herein Security ',
 'SOME THIS PAYMENT FOR MINISTRY BE PROCESS AMOUNT RECEIVE YOU ',
 'lord Lord church husband Blessed always Bible Christians coast Coast ',
 '7bit Content message 8859 iso Encoding charset Type Transfer may ',
 'TABLE TD SECURITY coast He junior brother STRONG security What ',
 'Charity 20 charity Now World organizations cared anymore Asia medical ',
 'br Charity span 3D charity

### word2vec

In [19]:
# word2vec

import gensim, logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

C:\Users\nems\Anaconda3\lib\site-packages\gensim\utils.py:862: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


In [20]:
# train word2vec on the two sentences

#model = gensim.models.Word2Vec(example_doc, min_count=1, size=200, hs=1, negative=0)
model = gensim.models.Word2Vec.load("word2vec.sav")

2017-10-29 06:53:33,314 : INFO : loading Word2Vec object from word2vec.sav
2017-10-29 06:53:33,321 : INFO : loading wv recursively from word2vec.sav.wv.* with mmap=None
2017-10-29 06:53:33,322 : INFO : setting ignored attribute syn0norm to None
2017-10-29 06:53:33,325 : INFO : setting ignored attribute cum_table to None
2017-10-29 06:53:33,327 : INFO : loaded word2vec.sav


In [28]:
letter_score = []
score = [0] * 10
for letter in key_list:
    if len(letter) == 0:
        pass
    else:
        score_ = model.score(letter.split())
        for i in range(10):
            score[i] += score_[i]
        #letter_score.append(score_.reshape(1, score_.shape[0]))

2017-10-29 07:13:34,443 : INFO : scoring sentences with 3 workers on 86 vocabulary and 200 features, using sg=0 hs=1 sample=0.001 and negative=0
2017-10-29 07:13:34,447 : INFO : reached end of input; waiting to finish 1 outstanding jobs
2017-10-29 07:13:34,449 : INFO : scoring 10 sentences took 0.0s, 2762 sentences/s
2017-10-29 07:13:34,451 : INFO : scoring sentences with 3 workers on 86 vocabulary and 200 features, using sg=0 hs=1 sample=0.001 and negative=0
2017-10-29 07:13:34,455 : INFO : reached end of input; waiting to finish 1 outstanding jobs
2017-10-29 07:13:34,457 : INFO : scoring 10 sentences took 0.0s, 2381 sentences/s
2017-10-29 07:13:34,459 : INFO : scoring sentences with 3 workers on 86 vocabulary and 200 features, using sg=0 hs=1 sample=0.001 and negative=0
2017-10-29 07:13:34,467 : INFO : reached end of input; waiting to finish 1 outstanding jobs
2017-10-29 07:13:34,468 : INFO : scoring 10 sentences took 0.0s, 1296 sentences/s
2017-10-29 07:13:34,470 : INFO : scoring se

2017-10-29 07:13:34,657 : INFO : reached end of input; waiting to finish 1 outstanding jobs
2017-10-29 07:13:34,661 : INFO : scoring 10 sentences took 0.0s, 1407 sentences/s
2017-10-29 07:13:34,663 : INFO : scoring sentences with 3 workers on 86 vocabulary and 200 features, using sg=0 hs=1 sample=0.001 and negative=0
2017-10-29 07:13:34,670 : INFO : reached end of input; waiting to finish 1 outstanding jobs
2017-10-29 07:13:34,672 : INFO : scoring 10 sentences took 0.0s, 1637 sentences/s
2017-10-29 07:13:34,674 : INFO : scoring sentences with 3 workers on 86 vocabulary and 200 features, using sg=0 hs=1 sample=0.001 and negative=0
2017-10-29 07:13:34,678 : INFO : reached end of input; waiting to finish 1 outstanding jobs
2017-10-29 07:13:34,680 : INFO : scoring 10 sentences took 0.0s, 2983 sentences/s
2017-10-29 07:13:34,682 : INFO : scoring sentences with 3 workers on 86 vocabulary and 200 features, using sg=0 hs=1 sample=0.001 and negative=0
2017-10-29 07:13:34,685 : INFO : reached en

IndexError: index 4 is out of bounds for axis 0 with size 4

In [34]:
kw_vec = np.array(score)/1000 ####
print (kw_vec)

[-0.59686035 -0.6725697  -0.69134709 -0.62265558 -0.59475399 -0.69835925
 -0.67224162 -0.67393852 -0.67712811 -0.58945206]
